In [1]:
import numpy as np # linear algebra import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) import pandas as pd import string import gensim import nltk from sklearn.metrics.pairwise import cosine_similarity nltk.download('stopwords') nltk.download('punkt')

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import os 
for dirname, _, filenames in os.walk('/kaggle/input'): 
    for filename in filenames: 
        print(os.path.join(dirname, filename))

!pip install sentence-transformers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__results__.html
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/vectorizedData.csv
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__notebook__.ipynb
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__output__.json
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/custom.css
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__results___files/__results___9_0.png
/kaggle/input/googlenewsvectors/GoogleNews-vectors-negative300.bin
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4deee0fe82777c2c7cc2951254ae49bd054485c90bed17ba2814f262e462c0fb
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-tran

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors


def similar_books(user_input):
    model_path = "/kaggle/input/googlenewsvectors/GoogleNews-vectors-negative300.bin"
    model = KeyedVectors.load_word2vec_format(model_path, binary=True)
    books = pd.read_csv("/kaggle/input/2-recommended-reads-conversion-of-data-to-num/vectorizedData.csv")

    # Clean and tokenize user input
    stop_words = stopwords.words('english')
    user_tokens = [token.lower() for token in word_tokenize(user_input) if token.isalpha() and token.lower() not in stop_words]
    cleaned_input = ' '.join(user_tokens)

    # Vectorize cleaned user input
    input_vector = np.zeros((model.vector_size,))
    count = 0
    for token in cleaned_input.split():
        if token in model:
            input_vector += model[token]
            count += 1
    if count > 0:
        input_vector /= count

    # Clean and tokenize book descriptions and vectorize
    book_vectors = []
    book_titles = []
    book_authors = []
    for title, author, desc in zip(books['booktitle'], books['authorname'], books['bookdescription']):
        desc_tokens = [token.lower() for token in word_tokenize(desc) if token.isalpha() and token.lower() not in stop_words]
        cleaned_desc = ' '.join(desc_tokens)
        if cleaned_desc:
            desc_vector = np.zeros((model.vector_size,))
            count = 0
            for token in cleaned_desc.split():
                if token in model:
                    desc_vector += model[token]
                    count += 1
            if count > 0:
                desc_vector /= count
                book_vectors.append(desc_vector)
                book_titles.append(title)
                book_authors.append(author)

    # Compute cosine similarities between input vector and book vectors
    similarities = cosine_similarity([input_vector], book_vectors)

    # Combine book data and similarity scores into DataFrame
    book_data = pd.DataFrame({'booktitle': book_titles, 'authorname': book_authors, 'bookdescription': books['bookdescription'][:len(book_vectors)], 'similarity': similarities[0]})

    # Remove duplicate books based on title and author
    book_data.drop_duplicates(subset=['booktitle', 'authorname'], inplace=True)

    # Sort by similarity and return top 5 books
    similar_books = book_data.sort_values(by='similarity', ascending=False).head(5)

    # Generate recommendation string
    recommendation = "Recommended books based on your input:\n"
    for index, row in similar_books.iterrows():
        recommendation += f"{row['booktitle']} by {row['authorname']} - Similarity: {round(row['similarity'], 3)}\n"

    print(recommendation)


In [3]:
user_input = "a space journey with friends"
similar_books(user_input)

Recommended books based on your input:
Field Trip to the Moon by John L. Hare - Similarity: 0.644
Space Chantey by R.A. Lafferty - Similarity: 0.643
Odyssey One by Evan Currie - Similarity: 0.621
A Love Through Time by Terri Brisbin - Similarity: 0.62
The Heart of Matter by Evan Currie - Similarity: 0.619



In [4]:
#drop same name books from data
